# SSNE - Mini projekt 2
## Przewidywanie kategorii cenowej mieszkań
### Wojtek Zarzecki, Janek Filipecki

Załóżmy, że chcemy kupić mieszkanie. Do dyspozycji mamy 100 000 dolarów, możemy też wziąć kredyt na kolejne 250 tysięcy, co da nam w sumie budżet w wysokości 350 000 dolarów. Stwórzmy model który pomoże nam przewidzieć, czy mieszkanie o pewnych, zadanych parametrach, możemy kupić za własne pieniądze (cheap), z kredytem (average), czy jest poza naszym zasięgiem (expensive).

W oparciu o dostępne atrybuty zbuduj model, który pomoże oszacować, czy dana nieruchomość należy do klasy cheap, average czy expensive. Do dyspozycji mają Państwo dane treningowe (train_data.csv) z oryginalnymi cenami nieruchomości (SalePrice), oraz, tak jak ostatnio, zbiór testowy (test_data.csv).

UWAGA Proszę dokładnie zastosować się do poniższej instrukcji
W ramach rozwiązania, proszę oddać poprzez Teamsy plik - archiwum .zip z kodem (w formie notebooka, lub skryptu/skryptów .py) oraz plikiem .csv z predykcjami na zbiorze test_data.csv.
W pliku z predykcjami powinna się znajdować dokładnie jedna kolumna, oznaczająca przewidywaną przez Państwa klasę ceny mieszkania (0 <- cheap, 1 <- average, 2 <- expensive).
Końcowe wyniki obliczał będę w oparciu o średnią dokładność dla każdej klasy. Proszę zwrócić uwagę na fakt, że klasy są mocno niezbalansowane!
Koniecznie proszę sprawdzić format zwracanych przez Państwa predykcji (tyle predykcji ile elementów w zbiorze testowym, brak nagłówków, jedna kolumna, itd.)
Bardzo proszę nazywać wszystkie pliki nazwiskami autorów (najlepiej alfabetycznie), lub nazwą zespołu. Nazwę głównego archiwum zip proszę dodatkowo rozpocząć od przedrostka poniedzialek_ lub piatek_ (nie pon/pia), proszę też nie umieszczać plików w dodatkowych podfolderach tylko bezpośrednio .
W MS ​Teams wszystkim przydzieliłem zadanie, ale bardzo proszę, żeby tylko jeden (dowolny) członek zespołu je zwrócił.
W razie pytań zapraszam do korespondencji lub na konsultacje.

In [84]:
import torch, torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
import numpy as np
import pandas as pd
import torch.utils.data as data
import seaborn as sns
import matplotlib.pyplot as plt

In [76]:
train_data = pd.read_csv("pakiet/train_data.csv")
test_data = pd.read_csv("pakiet/test_data.csv")

### Wstępna analiza danych

In [77]:
train_data.describe()

,SalePrice,YearBuilt,Size(sqf),Floor,N_Parkinglot(Ground),N_Parkinglot(Basement),N_manager,N_elevators,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
count,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000
mean,222177.477207,2002.977934,959.958778,11.994665,192.789040,572.857662,6.313773,11.055771,5.824200,9.860330,10.872696
std,106325.535526,8.765838,384.548456,7.581330,215.455916,408.179957,3.224556,7.717030,2.344331,3.444848,4.427445
min,34070.000000,1978.000000,135.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,144752.000000,1993.000000,644.000000,6.000000,11.000000,184.000000,5.000000,5.000000,4.000000,8.000000,7.000000
50%,209588.000000,2006.000000,910.000000,11.000000,100.000000,536.000000,6.000000,11.000000,5.000000,9.000000,10.000000
75%,291570.000000,2007.000000,1160.000000,17.000000,249.000000,798.000000,8.000000,16.000000,7.000000,13.000000,15.000000
max,585840.000000,2015.000000,2337.000000,43.000000,713.000000,1321.000000,14.000000,27.000000,10.000000,16.000000,17.000000


In [78]:
for c in train_data.columns:
    print("---- %s ---" % c)
    print(train_data[c].value_counts())

---- SalePrice ---
149274    38
75920     38
176991    38
265486    36
230088    35
          ..
227876     1
194247     1
328318     1
76106      1
557522     1
Name: SalePrice, Length: 841, dtype: int64
---- YearBuilt ---
2007    881
2005    512
1993    478
2006    471
1992    322
2013    307
2014    271
1986    244
2008    198
2009    164
2015     89
1997     77
1978     51
1985     47
2003      8
1980      4
Name: YearBuilt, dtype: int64
---- Size(sqf) ---
914    683
644    491
910    381
868    209
572    198
      ... 
715      1
291      1
163      1
277      1
839      1
Name: Size(sqf), Length: 89, dtype: int64
---- Floor ---
4     228
7     225
13    214
11    213
8     211
5     210
6     204
10    190
15    189
12    188
3     184
9     179
2     174
14    171
16    148
17    139
1     138
18    132
19    129
20    119
21     96
24     75
22     64
23     52
25     49
26     42
27     22
29     20
30     19
31     18
28     16
33     14
43      7
32      7
41      7
35     

Sprawdźmy współczynniki korelacji pomiędzy kolumnami a wartością przewidywaną

In [79]:
for c in train_data.drop("SalePrice", axis=1).columns:
    if np.issubdtype(train_data[c].dtypes, np.number):
        print("---- %s ---" % c)
        print(train_data["SalePrice"].corr(train_data[c]))
    else:
        print("---- %s ---" % c)
        print(train_data["SalePrice"].corr(train_data.apply(lambda x: pd.factorize(x)[0])[c]))

---- YearBuilt ---
0.4640763433231847
---- Size(sqf) ---
0.693544596400338
---- Floor ---
0.3310139015426135
---- HallwayType ---
-0.5481994375752809
---- HeatingType ---
-0.046392701503095206
---- AptManageType ---
-0.242499805985561
---- N_Parkinglot(Ground) ---
-0.14664694531885986
---- N_Parkinglot(Basement) ---
0.4815956913845701
---- TimeToBusStop ---
0.1630192048492829
---- TimeToSubway ---
0.13831003027899635
---- N_manager ---
0.36032429506201047
---- N_elevators ---
0.2038567831891158
---- SubwayStation ---
-0.04039269642864051
---- N_FacilitiesInApt ---
0.5098739740726335
---- N_FacilitiesNearBy(Total) ---
-0.42190499818357907
---- N_SchoolNearBy(Total) ---
-0.3785646726432567


Z powyższej analizy wynika że potencjalnie możemy pozbyć się kolumn "HeatingType", "SubwayStation"

In [80]:
sale_prices = train_data["SalePrice"]
expensive = sale_prices[sale_prices > 350000].count()
print("expensive: %s" % expensive)
average = sale_prices[sale_prices > 100000].count() - expensive
print("average: %s" % average)
cheap = len(sale_prices) - average - expensive
print("cheap: %s" % cheap)

expensive: 570
average: 2992
cheap: 562


Jak widać dataset jest istotnie dość niezbalansowany

In [82]:
train_data.dtypes

SalePrice                      int64
YearBuilt                      int64
Size(sqf)                      int64
Floor                          int64
HallwayType                   object
HeatingType                   object
AptManageType                 object
N_Parkinglot(Ground)         float64
N_Parkinglot(Basement)       float64
TimeToBusStop                 object
TimeToSubway                  object
N_manager                    float64
N_elevators                  float64
SubwayStation                 object
N_FacilitiesInApt              int64
N_FacilitiesNearBy(Total)    float64
N_SchoolNearBy(Total)        float64
dtype: object

### Encoding

In [114]:
def one_hot_encoder(df):
    columns = df.select_dtypes(include=object).columns
    output_df = df.copy()
    for c in columns:
        output_df = output_df.drop(c, axis=1).join(pd.get_dummies(train_data[c], c))
    return output_df

In [115]:
encoded_train, encoded_test = one_hot_encoder(train_data), one_hot_encoder(test_data)